In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps
import os

In [2]:

conn = ps.connect(
    host="localhost",
    port="5432",
    database="healthrecords",
    user="postgres",
    password='password'
)

# Create a cursor object
cur = conn.cursor()


In [3]:
# Function that creates a table in the database to store the following patient information:
# social security number, name, address, email, phonenumber and insurance ID as column headers.

def create_table(table_name, conn, cursor):
    try:
        query = "CREATE TABLE " + table_name + """(
                    SSN integer PRIMARY KEY,
                    Name VARCHAR(50) NOT NULL,
                    Address VARCHAR(200) NOT NULL,
                    Email VARCHAR(100) NOT NULL,
                    Phone VARCHAR(100) NOT NULL,
                    Insurance_id integer
                    )"""
        cursor.execute(query)
        conn.commit()
    except ps.Error as e:
        conn.rollback()  # Rollback the transaction in case of an error
        print("Error:", e)


In [4]:
create_table('patients', conn, cur)

Error: relation "patients" already exists



In [4]:
# Function to insert a patient into the table
def insert_patient(table_name, ssn, name, address, email, phone, insurance_id):
    try:
        query = f"INSERT INTO {table_name}(ssn, name, address, email, phone, insurance_id) VALUES({ssn}, '{name}', '{address}', '{email}', '{phone}', {insurance_id})"
        cur.execute(query)
        conn.commit()
    except ps.Error as e:
        conn.rollback()
        print("Error: ", e)

In [5]:
insert_patient('patients', 123343434, "John Smith", "123 Main st", "someting@email.com", "1234567890", 1234567890)

Error:  duplicate key value violates unique constraint "patients_pkey"
DETAIL:  Key (ssn)=(123343434) already exists.



In [6]:
import names
import random
from random_address import real_random_address
            
        

In [7]:
def create_test_sample(n):
    def rand_ssn(n):
        SSN_list = []
        for i in range(n):
            r = random.random()
            random_SSN = '{:.15f}'.format(r)  # random num between 0 & 1 to 15dp
            random_10_digits = random_SSN[-10:] # getting the last 10 digits of the random number
            SSN = random_10_digits
            SSN_list.append(int(SSN))
        return SSN_list
    
    def rand_full_name(n):
        full_name_list = []
        for i in range(n):
            name = names.get_full_name()
            full_name_list.append(name)
        return full_name_list
    
    def random_phone_number(n):
        phone_numbers_list = []
        for i in range(n):
            r = random.random()
            random_time = '{:.15f}'.format(r)  # creates random number between 0 and 1
            random_10_digits = random_time[-10:]
            phone_number = "07" + random_10_digits
            phone_numbers_list.append(str(phone_number))
        return phone_numbers_list
    
    def random_real_address(n):
        address_list = []
        for i in range(n):
            address = real_random_address()
            address_list.append(address['address1'])
        return address_list
        

    def e_mail_generator(list_of_names):
        e_mail_list = []
        for i in list_of_names:
            name = i
            e_mail_list.append(name.split(" ")[0] + "@emailaddress.com")
        return e_mail_list
    
    def random_insuranceid(n):
        insuranceid_list = []
        for i in range(n):
            r = random.random()
            random_nr = '{:.15f}'.format(r)   # creates random number between 0 and 1
            random_10_digits = random_nr[-10:]
            insuranceid = random_10_digits
            insuranceid_list.append(int(insuranceid))
        return insuranceid_list
    
    SSN_list = rand_ssn(n)
    name_list = rand_full_name(n)
    address_list = random_real_address(n)
    e_mail_list = e_mail_generator(name_list)
    phone_number_list = random_phone_number(n)
    insuranceid_list = random_insuranceid(n)
    
    # CHECK IF ALL LISTS ARE SAME LENGTH

    for i in [SSN_list, name_list, address_list, e_mail_list, phone_number_list, insuranceid_list]:
        if len(i) == n:
            pass
        else:
            raise Exception("length lists not equal")
            
    #PRINT FIRST RECORDS OF TEST SAMPLE
    #print("TEST SAMPLE COMPLETED, FIRST TWO RECORDS =")
    #for i in range(10):
        #print(SSN_list[i], name_list[i], address_list[i], e_mail_list[i], phone_number_list[i], insuranceid_list[i])
    
    print("")
    print("STARTING WITH POPULATING DATABASE")
    for i in range(n):
        insert_patient("Patients", SSN_list[i], name_list[i], address_list[i], e_mail_list[i], phone_number_list[i], insuranceid_list[i])

    print("")
    print("POPULATION COMPLETED")
    print("")


In [ ]:
# populating the patients database with 1000 patients
create_test_sample(1000)

In [8]:
#Putting all our data into a dataframe for viewing

table_name="Patients"
query = """SELECT * FROM """+table_name
cur.execute(query)
df = pd.DataFrame(cur.fetchall())
print(df)
    

             0                    1                              2  \
0    123343434           John Smith                    123 Main st   
1    128494285         Dallas Prado         46 Sugarloaf Hill Road   
2   1306029517         Ruth Burnham           2186 Knollwood Drive   
3    316957549         Tommy Thomas             1215 Joseph Avenue   
4   1624625131     Elizabeth Hooper              1772 150th Avenue   
5    453332955         Donald Moore  467 North Mountain Ranch Road   
6    758094984       Gregory Franco            105 Inglewood Drive   
7    601113569           Sarah Dunn                 301 East Court   
8    211118215         Tony Sawyers            6755 Mulberry Drive   
9    468149756         Lisa Simmons               136 Acacia Drive   
10  1875491657        Edgar Nolasco     3516 North Virginia Avenue   
11  1143106513   Antoinette Jackson           20270 Andover Avenue   
12   958689067         Julius Vella       165 New Hampshire Avenue   
13  1987834508      

In [9]:
# Function to find a patient

def find_a_patient(patient_full_name, table_name):
    query = f"SELECT * FROM {table_name} WHERE name = '{patient_full_name}'"
    print(query)
    cur.execute(query)
    row = cur.fetchone() # returns single row if available and None if not
    if row == None:
        print('No patient found')
        return
    else:
        print(f"SSN number for {patient_full_name}: {row[0]}")

In [10]:
find_a_patient('Dallas Prado', 'patients')

SELECT * FROM patients WHERE name = 'Dallas Prado'
SSN number for Dallas Prado: 128494285


In [11]:
#Finding a ptient from their adrress
def find_patient_from_address(address, table_name):
    query = f"SELECT * FROM {table_name} WHERE address = '{address}'"
    cur.execute(query)
    row = cur.fetchone() # returns single row if available and None if not
    if row == None:
        print('No patient found')
        return
    else:
        print(f"This address is for: {row[1]}")

In [12]:
find_patient_from_address('68 Route 66', 'patients')

This address is for: Mary Montalto


In [13]:
def find_patient_from_last_name(last_name, table_name):
    query = f"SELECT * FROM {table_name} WHERE name LIKE '%{last_name}'"
    cur.execute(query)
    row = cur.fetchone() # returns single row if available and None if not
    if row == None:
        print('No patient found')
        return
    else:
        print(f"Full name: {row[1]}\nSSN: {row[0]}")

In [14]:
find_patient_from_last_name('Nolasco', 'patients')

Full name: Edgar Nolasco
SSN: 1875491657


In [15]:
print(df.head(10)) #prints first 10 patients in data frame

            0                 1                              2  \
0   123343434        John Smith                    123 Main st   
1   128494285      Dallas Prado         46 Sugarloaf Hill Road   
2  1306029517      Ruth Burnham           2186 Knollwood Drive   
3   316957549      Tommy Thomas             1215 Joseph Avenue   
4  1624625131  Elizabeth Hooper              1772 150th Avenue   
5   453332955      Donald Moore  467 North Mountain Ranch Road   
6   758094984    Gregory Franco            105 Inglewood Drive   
7   601113569        Sarah Dunn                 301 East Court   
8   211118215      Tony Sawyers            6755 Mulberry Drive   
9   468149756      Lisa Simmons               136 Acacia Drive   

                            3             4           5  
0          someting@email.com    1234567890  1234567890  
1     Dallas@emailaddress.com  075090876400  2070722123  
2       Ruth@emailaddress.com  070538154074  1443364608  
3      Tommy@emailaddress.com  0796451422

In [16]:
print(df.dtypes) #prints datatypes of each column

0     int64
1    object
2    object
3    object
4    object
5     int64
dtype: object


In [17]:
print(df.columns) # column range

RangeIndex(start=0, stop=6, step=1)


In [18]:
print(df.shape[0], df.shape[1]) # size of dataframe 48 rows and 6 columns

48 6


In [19]:
user_input_name = input('Please enter the patient\'s full name')


Please enter the patient's full nameJohn Smith


In [20]:
def user_search(name):
    for index, row in df.iterrows():
        if row[1] == name:
            patient_name = row[1]
            patient_ssn = row[0]
            patient_address = row[2]
            patient_email = row[3]
            patient_number = row[4]
            patient_insuranceid = row[5]
    print(patient_name, patient_ssn, patient_address, patient_email, patient_number, patient_insuranceid)

In [21]:
user_search(user_input_name)

John Smith 123343434 123 Main st someting@email.com 1234567890 1234567890


In [22]:
def editDistance(str1, str2, m, n):

    # If first string is empty, the only option is to
    # insert all characters of second string into first
    if m == 0:
        return n

    # If second string is empty, the only option is to
    # remove all characters of first string
    if n == 0:
        return m

    # If last characters of two strings are same, nothing
    # much to do. Ignore last characters and get count for
    # remaining strings.
    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)

    # If last characters are not same, consider all three
    # operations on last character of first string, recursively
    # compute minimum cost for all three operations and take
    # minimum of three values.
    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert
                   editDistance(str1, str2, m-1, n),    # Remove
                   editDistance(str1, str2, m-1, n-1)    # Replace
                   )


In [25]:
given_name = input('Who do you want to search for? ')

def find_closest_name(given_name):
    for index, row in df.iterrows():
        min_distance = float('Inf')
        db_name = str(row[1])
        distance = editDistance(given_name, db_name, len(given_name), len(db_name))
        if distance < min_distance:
            min_distance = distance
            chosen_name = row[1]
            chosen_name_email = row[3]
    
    print('Patient name is: ' + chosen_name + ' and you can contact them here: ' + chosen_name_email)

Who do you want to search for? johnny


In [26]:
find_closest_name(given_name)

Patient name is: Robert Williams and you can contact them here: Robert@emailaddress.com
